### Exponential Dispersal Function

&nbsp;

An exponential dispersal function describes the spatiotemporal dispersal of wildlife. The exponential function represents how the density decreases as you move away from its source. The model in this script is listed below.

$$Y = p \times \mathrm{e}^{-\frac{X}{b}}$$

where

Y denotes the probability of dispersal at a distance X from the source

X denotes the distance from the source

p denotes the proportion which does not disperse and stays at the source (X = 0)

b denotes the average dispersal distance

In [1]:
import os
import pandas as pd
from pyproj import Proj
import numpy as np
import pyodbc
import itertools
from scipy.optimize import curve_fit
os.chdir('C:/Users/tm/Downloads/utas/thesis/chapter1/tasman')

### funcs

In [2]:
#gompertz curve equation
def dispersal_func(X,p,b):
    return p*np.exp(-X/b)

#using mle to estimate
def get_func_params(x,y):
    popt,pcov=curve_fit(dispersal_func,x,y,p0=(0.5,np.mean(x)))
    return popt

In [3]:
#coordinates conversion
def convert_easting_northing_to_lat_lon(easting, northing,):
    proj = Proj('+proj=utm +zone=55 +south +ellps=GRS80 +units=m +no_defs')
    lon, lat = proj(easting, northing, inverse=True)
    return lat, lon

#spherical distance computation by chatgpt
def haversine_distance(lat1, lon1, lat2, lon2):
    
    # Convert degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Earth's radius in kilometers
    radius = 6371

    # Difference in latitudes and longitudes
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = np.sin(delta_lat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate distance
    distance = radius * c

    return distance

### cleanse

In [4]:
dataset=[]
for i in os.listdir('./Data collation for Forestier Peninsula'):
    if 'xlsx' not in i:
        filename=os.listdir(f'./Data collation for Forestier Peninsula/{i}')[0]
        spreadsheet=pd.ExcelFile(f'./Data collation for Forestier Peninsula/{i}/{filename}')        
        dataset.append(spreadsheet.parse(spreadsheet.sheet_names[0]))

C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default sty

In [5]:
df=pd.concat([i for i in dataset])

In [6]:
#convert coordinates
df['lat'],df['lon']=convert_easting_northing_to_lat_lon(df['EASTING'], df['NORTHING'])

C:\Users\tm\AppData\Local\Temp\ipykernel_15892\2425016858.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['lat'],df['lon']=convert_easting_northing_to_lat_lon(df['EASTING'], df['NORTHING'])
C:\Users\tm\AppData\Local\Temp\ipykernel_15892\2425016858.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['lat'],df['lon']=convert_easting_northing_to_lat_lon(df['EASTING'], df['NORTHING'])


In [7]:
lonmax=148.19770312099925;latmax=-42.826341401892726;
lonmin=147.5879619192109;latmin=-43.27791719967915;
lat_centre=-43.03648678978083; lon_centre=147.83240771632424;
gridlen=50

#locate where the devils are on a 20 by 20 raster
lonunit=(lonmax-lonmin)/gridlen
latunit=(latmax-latmin)/gridlen
df['lat num']=(df['lat']-latmin)//latunit
df['lon num']=(df['lon']-lonmin)//lonunit
df['coordinates']=list(zip(df['lat num'].astype(int),df['lon num'].astype(int)))

C:\Users\tm\AppData\Local\Temp\ipykernel_15892\1004644662.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['lat num']=(df['lat']-latmin)//latunit
C:\Users\tm\AppData\Local\Temp\ipykernel_15892\1004644662.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['lon num']=(df['lon']-lonmin)//lonunit
C:\Users\tm\AppData\Local\Temp\ipykernel_15892\1004644662.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

In [8]:
#remove devils outside of maria island
removal=[]
for loc in df['coordinates'].unique():
    if loc[0]>gridlen or loc[0]<0 or loc[1]>gridlen or loc[1]<0:
        removal.append(loc)

df=df[~df['coordinates'].isin(removal)]

In [9]:
#sort by date
grande=df[['INDIVIDUAL','OBSERVATION_DATE','FIELD_TRIP','lat','lon','coordinates']].sort_values(['INDIVIDUAL','OBSERVATION_DATE'])

#datetimeindex
grande['OBSERVATION_DATE']=pd.to_datetime(grande['OBSERVATION_DATE'])

In [10]:
#work on non disease natural growth
grande['year']=grande['OBSERVATION_DATE'].dt.year
grande=grande[grande['year']>=2015]

In [11]:
grande=grande[~grande['INDIVIDUAL'].isnull()]

In [12]:
#only preserves animals that have been recaptured more than 300 days apart
target_ids=[]
for i in grande['INDIVIDUAL'].unique():
    subset=grande[grande['INDIVIDUAL']==i].copy()
    if len(subset)==1:
        continue
    counting=(subset['OBSERVATION_DATE'].iloc[-1]-subset['OBSERVATION_DATE'].iloc[0]).days
    if counting<300:
        continue
    target_ids.append(i)
freq=grande[grande['INDIVIDUAL'].isin(target_ids)]

### compute X

In [13]:
#compute the distance of annual dispersal
output=pd.DataFrame(columns=['trip id', 'trip range', 'devil id', 'distance','trip coordinates'])

for i in freq['INDIVIDUAL'].unique():
    subset=freq[freq['INDIVIDUAL']==i].copy()
    combs=list(itertools.combinations(subset['OBSERVATION_DATE'],2))

    #for each trip, only takes the first date of captured
    for ind,val in enumerate(combs):
        dif=(val[1]-val[0]).days
        if dif>330 and dif<390:
            id0=subset['FIELD_TRIP'][subset['OBSERVATION_DATE']==val[0]].iloc[0]
            id1=subset['FIELD_TRIP'][subset['OBSERVATION_DATE']==val[1]].iloc[0]
            combs[ind]=[str(id0)+'-'+str(id1),val]            
    result=[j for j in combs if type(j)==list]
    
    dataset=pd.DataFrame(result,columns=['trip id','trip range'])
    dataset=dataset.loc[dataset['trip id'].drop_duplicates().index]
    dataset['devil id']=i

    #compute spherical distance
    arr1=[]
    arr2=[]
    arr3=[]
    for k in dataset.index:
        startdate=dataset['trip range'].loc[k][0]
        enddate=dataset['trip range'].loc[k][1]

        lat1=subset['lat'][subset['OBSERVATION_DATE']==startdate].iloc[0]
        lon1=subset['lon'][subset['OBSERVATION_DATE']==startdate].iloc[0]
        lat2=subset['lat'][subset['OBSERVATION_DATE']==enddate].iloc[0]
        lon2=subset['lon'][subset['OBSERVATION_DATE']==enddate].iloc[0]
        grid1=subset['coordinates'][subset['OBSERVATION_DATE']==startdate].iloc[0]
        grid2=subset['coordinates'][subset['OBSERVATION_DATE']==enddate].iloc[0]

        arr1.append(haversine_distance(lat1, lon1, lat2, lon2))
        arr2.append((lat1, lon1, lat2, lon2))
        arr3.append((grid1,grid2))
    dataset['distance']=arr1
    dataset['trip coordinates']=arr2
    dataset['trip grid']=arr3
    
    output=pd.concat([output,dataset])

output.reset_index(inplace=True,drop=True)

### compute Y

In [14]:
#keep one devil per trip
raw=grande.sort_values([ 'FIELD_TRIP', 'OBSERVATION_DATE'])
raw.reset_index(inplace=True,drop=True)
raw=raw.loc[raw[['INDIVIDUAL','FIELD_TRIP','lat','lon']].drop_duplicates().index]

#find starting point
raw['de']=raw['coordinates']

#find starting point
output['de']=output['trip grid'].apply(lambda x:x[0])

In [15]:
# #count the devils recaptured at each starting point
# numerator=output.groupby('trip grid').count()[['trip id']]
# numerator['de']=[i[0] for i in numerator.index]
# numerator['grid dist']=pd.Series(numerator.index).apply(lambda x: ((x[0][0]-x[1][0])**2+(x[0][1]-x[1][1])**2)**0.5).tolist()

# #count the devils captured at each starting point
# denominator=raw.groupby('de').count()[['lat']]
# denominator.reset_index(inplace=True)

# #compute probability
# prob=numerator.merge(denominator,on='de',how='left')
# prob.index=numerator.index
# prob['prob']=prob['trip id']/len(output)

# #merge
# prob.reset_index(inplace=True)
# prob=prob[['trip grid','prob']]

In [16]:
#compute grid distance
output['grid dist']=output['trip grid'].apply(lambda x: ((x[0][0]-x[1][0])**2+(x[0][1]-x[1][1])**2)**0.5)

In [17]:
#compute the probability of grid distance
prob=output.groupby(['grid dist']).count()[['trip id']]
prob['trip id']=prob['trip id']/prob['trip id'].sum()
prob.columns=['prob']
prob.reset_index(inplace=True)

In [18]:
#merge
output=output.merge(prob,on='grid dist',how='left')

In [19]:
#convert km to m
output['distance']*=1000

In [20]:
#result
get_func_params(output['distance'].tolist(),output['prob'].tolist())

array([4.15627440e-01, 3.66888484e+03])